<a href="https://colab.research.google.com/github/Inceptix/Cassava-leaf-disease-classification/blob/master/Code/Data_Acquisition_and_Understanding/Cassava_Disease_TFRecords_Structure_BaseModel_Confusion_Matrix_V01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Reference**
https://www.kaggle.com/ryanholbrook/tfrecords-basics

Setting up the environment

In [ ]:
import math, re, os
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from functools import partial
from sklearn.model_selection import train_test_split
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.4.1


Setting up TPUs

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)

Device: grpc://10.86.16.218:8470
INFO:tensorflow:Initializing the TPU system: grpc://10.86.16.218:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.86.16.218:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Number of replicas: 8


In [ ]:
# need to load from a google cloud bucket or you will get
# File system scheme '[local]' error
# this might change over time so need to go back to getting started notebook: https://www.kaggle.com/thucduong/getting-started-tpus-cassava-leaf-disease/edit
# and get the path again
GCS_PATH = 'gs://kds-9254a9aba4b37289f07bbbbe4e2af5c952dd2d811a2c823340ca8091'

In [ ]:
TRAINING_PATH = GCS_PATH + '/train_tfrecords/ld_train*.tfrec' 
TESTING_PATH = GCS_PATH + '/test_tfrecords/ld_test*.tfrec'
TRAINING_FILENAMES, VALID_FILENAMES = train_test_split(
    tf.io.gfile.glob(TRAINING_PATH),
    test_size=0.35, random_state=5
)

TEST_FILENAMES = tf.io.gfile.glob(TESTING_PATH)
print(TEST_FILENAMES)

['gs://kds-9254a9aba4b37289f07bbbbe4e2af5c952dd2d811a2c823340ca8091/test_tfrecords/ld_test00-1.tfrec']


All the data for this competition is in TF records so we need to decode these TFrecords file in order to get back the (image, label) pair. We can go from TFrecords files to (image, label) pair

Step 1: Create a function to read TF records and de-serialize these byte strings.

In [ ]:
def read_tfrecord(example, labeled):
    tfrecord_format = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.int64)
    } if labeled else {
        "image": tf.io.FixedLenFeature([], tf.string),
        "image_name": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example['image'])
    if labeled:
        label = tf.cast(example['target'], tf.int32)
        return image, label
    idnum = example['image_name']
    return image, idnum

**example** here means a (image, label) pair. 

Each example is a byte string that represents an image and a label (called target here) that represents the label [0,1,2,3,4]. But we need a method to convert this string into a data structure in which we can extract the info for the image and its label. 

This data structure is a dictionary.

In some academic literature, for each tfrecord, it's also called a serialized string. 

We use tf.io.parse_single_example(example, tf_record_format) to **de-serialize** the string back into a dictionary.

This dictionary has 2 key-value pairs in which the 2 keys are image and target. The dictionary key-value pairs vary depending on the serialized strings and how the original dataset was encoded. Some dictionaries for some datasets can have 3 or more key value pairs like (image, label, class_name).

In short, each example or each tf_record is a serialized string say "dffgjkshgsj24fjgjks43454" then we use:
tf.io.parse_single_example("dffgjkshgsj24fjgjks43454", tf_record_format) 

That gives us back the following dict:

features {

  feature {

    key: "image",
    value: {
      bytes_list {
        value: "\010\003\022\010\022\002\010\003\022\002\010\003\"$\000\000\000\000\001\000\000\000\002\000\000\000\003\000\000\000\004\000\000\000\005\000\000\000\006\000\000\000\007\000\000\000\010\000\000\000"
      }
    }

  }

  feature {

    key: "target",
    value: {
      int64_list {
        value: 0
      }
    }

  }

}

It looks quite confusing but just remember that it's a dict after de-serialization.

See more in tf.Example section of the notebook in Reference section above (reference notebook in the first cell) 

Step 2: Create a function that decode the image's byte lists into an actual image itself.
You might have notice that we have a function called decode_image() in our read_tfrecord() function above. We will define this function next!

In [ ]:
IMAGE_SIZE = [512, 512]

# decode the images
def decode_image(image):
    # decode with 3 channels (RGB)
    image = tf.image.decode_jpeg(image, channels=3)
    # normalize so each image's pixels are from 0 - 255 to 0 - 1 range
    image = tf.cast(image, tf.float32) / 255.0
    # apply the normalized pixels value on the images
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    return image

Step 3: Create a function to read TF records from multiple TFrecord files! This allows us to speed up the time to read multiple files.

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

def load_dataset(filenames, labeled=True, ordered=False):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTOTUNE) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(partial(read_tfrecord, labeled=labeled), num_parallel_calls=AUTOTUNE)
    return dataset

Summary: All TFrecord files -> load_dataset -> read_tfrecord -> decode_image -> get the actual image and its label

(a graphic maybe helpful here)

Step 4: Create a function to get the training/testing/validating datasets that we will feed directly into our model. Of course, this notebook is to explore the dataset structure not building a model.

This function of course shuffles and batches the (image, label) pairs for training.


In [ ]:
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

def get_training_dataset():
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)  

    # trying out different data augmentation techniques
    # dataset = dataset.map(data_augment, num_parallel_calls=AUTOTUNE) 
    # for technique in data_augment_sequence:
    #  dataset = dataset.map(technique, num_parallel_calls=AUTOTUNE) 

    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

def get_validation_dataset(ordered=False):
    dataset = load_dataset(VALID_FILENAMES, labeled=True, ordered=ordered) 
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

ALL TRAINING_FILENAMES, VALID_FILENAMES, TEST_FILENAMES are defined above when we were getting the file names from the Google Cloud Bucket.

Step 5: Build the model with LR scheduler and RESNET 50 as our base model. We use transfer learning.

In [ ]:
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-5, 
    decay_steps=10000, 
    decay_rate=0.9)

In [ ]:
CLASSES = ['0', '1', '2', '3', '4']
with strategy.scope():       
    img_adjust_layer = tf.keras.layers.Lambda(tf.keras.applications.resnet50.preprocess_input, input_shape=[*IMAGE_SIZE, 3])
    
    base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False)
    base_model.trainable = False
    
    model = tf.keras.Sequential([
        tf.keras.layers.BatchNormalization(renorm=True),
        img_adjust_layer,
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(8, activation='relu'),
        #tf.keras.layers.BatchNormalization(renorm=True),
        tf.keras.layers.Dense(len(CLASSES), activation='softmax')  
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_scheduler, epsilon=0.001),
        loss='sparse_categorical_crossentropy',  
        metrics=['sparse_categorical_accuracy'])

94773248/94765736 [==============================] - 1s 0us/step


Step 6: Train the model!

In [ ]:
# load data
train_dataset = get_training_dataset()
valid_dataset = get_validation_dataset()

In [ ]:
def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

In [ ]:
NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
NUM_VALIDATION_IMAGES = count_data_items(VALID_FILENAMES)

STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
VALID_STEPS = NUM_VALIDATION_IMAGES // BATCH_SIZE
EPOCHS = 10

history = model.fit(train_dataset, 
                    steps_per_epoch=STEPS_PER_EPOCH, 
                    epochs=EPOCHS,
                    validation_data=valid_dataset,
                    validation_steps=VALID_STEPS)

Epoch 1/10
104/104 [==============================] - 91s 640ms/step - loss: 1.5527 - sparse_categorical_accuracy: 0.2221 - val_loss: 1.3196 - val_sparse_categorical_accuracy: 0.5776
Epoch 2/10
104/104 [==============================] - 55s 529ms/step - loss: 1.3241 - sparse_categorical_accuracy: 0.5891 - val_loss: 1.3100 - val_sparse_categorical_accuracy: 0.6045
Epoch 3/10
104/104 [==============================] - 55s 530ms/step - loss: 1.2827 - sparse_categorical_accuracy: 0.6157 - val_loss: 1.2622 - val_sparse_categorical_accuracy: 0.6071
Epoch 4/10
104/104 [==============================] - 55s 527ms/step - loss: 1.2476 - sparse_categorical_accuracy: 0.6111 - val_loss: 1.2252 - val_sparse_categorical_accuracy: 0.6077
Epoch 5/10
104/104 [==============================] - 55s 530ms/step - loss: 1.2164 - sparse_categorical_accuracy: 0.6102 - val_loss: 1.1956 - val_sparse_categorical_accuracy: 0.6086
Epoch 6/10
104/104 [==============================] - 55s 532ms/step - loss: 1.1798 -

We can experiment with changining the EPOCHS number to see if accuracy improves!

Step 7: Evaluating the model! 

Try using confusion matrix

Reference: https://www.geeksforgeeks.org/python-tensorflow-math-confusion_matrix/

So we will try producing a labels array and a predictions array for the training data.

Since our model's output for each training data is the probabilities of that example belonging to each class, we will pick the class with the highest probability and make that class as our prediction.

In [ ]:
# this code will convert our test image data to a float32 
def to_float32(image, label):
    return tf.cast(image, tf.float32), label

In [ ]:
valid_ds = get_validation_dataset(ordered=True) 
print(valid_ds)

# https://stackoverflow.com/questions/56226621/how-to-extract-data-labels-back-from-tensorflow-dataset
# In case your tf.data.Dataset is batched, the following code will retrieve all the y labels:
validation_labels = np.concatenate([y for x, y in valid_ds], axis=0)

<PrefetchDataset shapes: ((None, 512, 512, 3), (None,)), types: (tf.float32, tf.int32)>


In [ ]:
NUM_VALIDATION_IMAGES = count_data_items(VALID_FILENAMES)

print('Dataset: {} validation images'.format(NUM_VALIDATION_IMAGES))

Dataset: 8017 validation images


In [ ]:
print(len(validation_labels))

8017


In [ ]:
print('Computing predictions on validation sets ...')
valid_images_ds = valid_ds
valid_images_ds = valid_images_ds.map(lambda image, idnum: image)
probabilities = model.predict(valid_images_ds)
predictions = np.argmax(probabilities, axis=-1)
print(predictions)

Computing predictions on validation sets ...
[3 3 3 ... 3 3 3]


In [ ]:
print(len(predictions))

8017


In [ ]:
# Evaluating confusion matric 
from sklearn.metrics import confusion_matrix

res = confusion_matrix(validation_labels,predictions) 
  
# Printing the result 
print('Confusion_matrix: ', res) 

Confusion_matrix:  [[   0   11    4  363   14]
 [   0   25   21  767   10]
 [   0    8   18  896    6]
 [   0    8    6 4851   11]
 [   0   23   33  923   19]]


In [ ]:
acc = np.sum(res.diagonal()) / np.sum(res)
print('Overall accuracy: {} %'.format(acc*100))

Overall accuracy: 61.282275165273795 %


In [ ]:
res

array([[   0,   11,    4,  363,   14],
       [   0,   25,   21,  767,   10],
       [   0,    8,   18,  896,    6],
       [   0,    8,    6, 4851,   11],
       [   0,   23,   33,  923,   19]])